In [92]:
import tensorflow as tf
import keras
import sklearn
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("scikit-learn:", sklearn.__version__)

TensorFlow: 2.16.1
Keras: 3.3.3
scikit-learn: 1.2.2


In [94]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [96]:
import os
dataset_path = '/kaggle/input/robin-base'
dataset_class = os.listdir(dataset_path)

train_dir = '/kaggle/input/robin-base/train'
val_dir = '/kaggle/input/robin-base/val'
test_dir = '/kaggle/input/robin-base/test'

In [97]:
for split_dir in [train_dir, val_dir, test_dir]:
    print(f"Contents of {split_dir}:")
    for class_dir in os.listdir(split_dir):
        class_path = os.path.join(split_dir, class_dir)
        if os.path.isdir(class_path):
            print(f" - Class: {class_dir}, Number of images: {len(os.listdir(class_path))}")

Contents of /kaggle/input/robin-base/train:
 - Class: elektronik, Number of images: 2099
 - Class: logam, Number of images: 2099
 - Class: plastik, Number of images: 2099
 - Class: buah_sayuran, Number of images: 2099
 - Class: makanan, Number of images: 2099
 - Class: daun, Number of images: 2099
 - Class: tekstil, Number of images: 2099
 - Class: medis, Number of images: 2099
 - Class: kaca, Number of images: 2099
 - Class: kertas, Number of images: 2099
Contents of /kaggle/input/robin-base/val:
 - Class: elektronik, Number of images: 600
 - Class: logam, Number of images: 600
 - Class: plastik, Number of images: 600
 - Class: buah_sayuran, Number of images: 600
 - Class: makanan, Number of images: 600
 - Class: daun, Number of images: 600
 - Class: tekstil, Number of images: 600
 - Class: medis, Number of images: 600
 - Class: kaca, Number of images: 600
 - Class: kertas, Number of images: 600
Contents of /kaggle/input/robin-base/test:
 - Class: elektronik, Number of images: 301
 - 

In [98]:
data = []

for class_name in dataset_class:
    for file in os.listdir(os.path.join(dataset_path, class_name)):
        data.append((os.path.join(dataset_path, class_name, file), class_name))

df = pd.DataFrame(data, columns=['filepath', 'label'])
df.head()

,filepath,label
0,/kaggle/input/robin-base/val/elektronik,val
1,/kaggle/input/robin-base/val/logam,val
2,/kaggle/input/robin-base/val/plastik,val
3,/kaggle/input/robin-base/val/buah_sayuran,val
4,/kaggle/input/robin-base/val/makanan,val


In [100]:
from tensorflow.keras.applications.efficientnet import preprocess_input

# Membuat ImageDataGenerator untuk augmentasi gambar pada data pelatihan
train_datagen = ImageDataGenerator(
    rotation_range=30,                  
    width_shift_range=0.10,           
    height_shift_range=0.10,            
    zoom_range=0.15,        
    horizontal_flip=True,              
    vertical_flip=False,             
    shear_range=0.05,                   
    brightness_range=[0.9, 1.1],      
    channel_shift_range=5,              
    fill_mode='nearest',
    preprocessing_function=preprocess_input 
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input 
)


In [101]:
image_size = (224, 224)
batch_size = 64

In [102]:
# Memuat dataset pelatihan, validasi, dan pengujian
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    shuffle=False,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    seed = 42
)

val_dataset = val_test_datagen.flow_from_directory(
    val_dir,
    shuffle=False,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    seed = 42
)

test_dataset = val_test_datagen.flow_from_directory(
    test_dir,
    shuffle=False,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    seed = 42
)

Found 20990 images belonging to 10 classes.
Found 6000 images belonging to 10 classes.
Found 3010 images belonging to 10 classes.


In [103]:
print(f"Number of batches in train_generator: {len(train_dataset)}")
print(f"Number of batches in val_generator: {len(val_dataset)}")

Number of batches in train_generator: 328
Number of batches in val_generator: 94


In [104]:
train_dataset.class_indices

{'buah_sayuran': 0,
 'daun': 1,
 'elektronik': 2,
 'kaca': 3,
 'kertas': 4,
 'logam': 5,
 'makanan': 6,
 'medis': 7,
 'plastik': 8,
 'tekstil': 9}

In [109]:
# Memuat EfficientNetB0 tanpa lapisan atas (top), dengan pretrained weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Mengunci semua lapisan base_model agar tidak dilatih
base_model.trainable = False

# Input Layer
inputs = layers.Input(shape=(224, 224, 3))

# Base Model
x = base_model(inputs)

# Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)

# Dense Layers
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Output Layer
outputs = layers.Dense(len(train_dataset.class_indices), activation='softmax')(x)

# Membuat model final
model = models.Model(inputs, outputs)



# Menentukan optimizer dan kompilasi model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Melihat ringkasan model
model.summary()


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_15     │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,839,341 (18.46 MB)

 Trainable params: 789,770 (3.01 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [110]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


# Menambahkan EarlyStopping, ModelCheckpoint, dan ReduceLROnPlateau
checkpoint = ModelCheckpoint(
    'robin_best_model.keras',  # Nama file untuk menyimpan model terbaik
    monitor='val_loss',  # Memantau val_loss
    save_best_only=True,  # Simpan hanya model terbaik
    mode='min',  # Minimalkan val_loss
    verbose=1  # Tampilkan informasi saat model disimpan
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor val_loss
    factor=0.5,  # Kurangi learning rate dengan faktor 0.5
    patience=10,  # Jika tidak ada perbaikan dalam 10 epoch, kurangi learning rate
    min_lr=0.00001,  # Learning rate tidak boleh lebih kecil dari 0.00001
    verbose=1  # Tampilkan informasi saat learning rate dikurangi
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor val_loss
    mode='min',  # Hentikan pelatihan jika val_loss tidak membaik
    patience=10,  # Hentikan jika tidak ada perbaikan selama 15 epoch
    restore_best_weights=True,  # Kembalikan bobot terbaik
    verbose=1  # Tampilkan informasi saat pelatihan dihentikan
)

# Gabungkan semua callback
callbacks = [
    checkpoint,
    reduce_lr,
    early_stopping
]

In [111]:
history = model.fit(
    train_dataset,  # Dataset pelatihan
    epochs=10, # Tentukan jumlah epoch sesuai kebutuhan
    validation_data=val_dataset,  # Dataset validasi
    callbacks=callbacks,  # Menambahkan callbacks
    steps_per_epoch = train_dataset.samples // train_dataset.batch_size,
    validation_steps = val_dataset.samples // val_dataset.batch_size   # Jumlah batch dalam validasi
)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 849ms/step - accuracy: 0.2421 - loss: 2.1292
Epoch 1: val_loss improved from inf to 1.07943, saving model to robin_best_model.keras
327/327 ━━━━━━━━━━━━━━━━━━━━ 331s 905ms/step - accuracy: 0.2425 - loss: 2.1283 - val_accuracy: 0.6712 - val_loss: 1.0794 - learning_rate: 1.0000e-04
Epoch 2/10
  1/327 ━━━━━━━━━━━━━━━━━━━━ 17s 55ms/step - accuracy: 0.2031 - loss: 2.1208

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss improved from 1.07943 to 0.55010, saving model to robin_best_model.keras
327/327 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.2031 - loss: 2.1208 - val_accuracy: 0.9375 - val_loss: 0.5501 - learning_rate: 1.0000e-04
Epoch 3/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 805ms/step - accuracy: 0.6022 - loss: 1.2088
Epoch 3: val_loss did not improve from 0.55010
327/327 ━━━━━━━━━━━━━━━━━━━━ 281s 843ms/step - accuracy: 0.6023 - loss: 1.2087 - val_accuracy: 0.7685 - val_loss: 0.7215 - learning_rate: 1.0000e-04
Epoch 4/10
  1/327 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - accuracy: 0.9688 - loss: 0.2101
Epoch 4: val_loss improved from 0.55010 to 0.47247, saving model to robin_best_model.keras
327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9688 - loss: 0.2101 - val_accuracy: 0.8958 - val_loss: 0.4725 - learning_rate: 1.0000e-04
Epoch 5/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 801ms/step - accuracy: 0.6905 - loss: 0.9424
Epoch 5: val_loss did not improve from 0.47247
327/327 ━━━━━━━━━━━

In [112]:
# Evaluasi model dengan data validasi
val_loss, val_accuracy = model.evaluate(val_dataset)

# Jika menggunakan data uji
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 12s 127ms/step - accuracy: 0.8175 - loss: 0.5665
48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 229ms/step - accuracy: 0.8318 - loss: 0.5468
Validation Loss: 0.5918298959732056
Validation Accuracy: 0.809499979019165
